# Chainer の学習と推論を SageMaker のローカルモードで行う

#### ノートブックに含まれる内容

- Chainer の学習と推論を SageMaker のローカルモードで行うやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: MNIST
- データ: MLP

In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

まず，ローカル実行に必要なコンポーネントをインストールします．セットアップ手順はシェルスクリプトにまとまっているので，これを実行します．

In [ ]:
!/bin/bash ./setup.sh

## データのロード

ここでは，`Chainer` でサポートされている関数を使って，MNIST データをダウンロードします．SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，ローカルに落とした MNIST データを npz 形式で固めてから，SageMaker のラッパー関数を使って S3 にアップロードします．

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．`upload_data()` メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することが可能です

In [ ]:
import chainer

train, test = chainer.datasets.get_mnist()

以下を実行する前に，**<span style="color: red;">2 箇所ある `notebook/chainer/XX/mnist` の XX を指定された適切な数字に変更</span>**してください．

In [ ]:
import os
import shutil
import numpy as np

train_images = np.array([data[0] for data in train])
train_labels = np.array([data[1] for data in train])
test_images = np.array([data[0] for data in test])
test_labels = np.array([data[1] for data in test])

try:
    os.makedirs('/tmp/data/train')
    os.makedirs('/tmp/data/test')

    np.savez('/tmp/data/train/train.npz', images=train_images, labels=train_labels)
    np.savez('/tmp/data/test/test.npz', images=test_images, labels=test_labels)

    train_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'train'),
        key_prefix='notebook/chainer/XX/mnist')
    test_input = sagemaker_session.upload_data(
        path=os.path.join('/tmp/data', 'test'),
        key_prefix='notebook/chainer/XX/mnist')
finally:
    shutil.rmtree('/tmp/data')

## Chainer を SageMaker で使う際のポイント

学習については，main 関数内にそのままモデルを記述するだけで大丈夫です．あとは SageMaker 側で学習ジョブを走らせる際に，main 関数内の学習処理が実行されます．ですので，既存の Chainer の実行スクリプトをほぼそのまま SageMaker に移植することができます．また，環境変数経由で入力データの場所や GPU の数などを取得することが可能です．これは `argparse` 経由で `main` 関数内で受け取ることができます．詳細は[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/chainer/README.rst)をご覧ください．

また推論時の処理は，`model_fn` で学習済みモデルをロードする部分だけ記述する必要があります．その他オプションで，前処理，推論処理，後処理部分を `input_fn`, `predict_fn`, `output_fn` で書くこともできます．デフォルトでは，`application/x-npy` コンテントタイプで指定される，NumPy 配列を入力として受け取ります． 


In [ ]:
!pygmentize 'chainer_mnist.py'

## モデルの学習を実行

以下のように，`Estimator` クラスの子クラスの `Chainer` オブジェクトを作成し，`fit()` メソッドで学習ジョブを実行します． `entry_point` で指定したローカルのスクリプトが，学習用のコンテナ内で実行されます．また合わせてローカルの `source_dir` を指定することで，依存するスクリプト群をコンテナにコピーして，学習時に使用することが可能です．

またここでは，インスタンス対応を `'local'` とすることにより，学習用のコンテナをノートブックインスタンスに落としてきて，このインスタンス内で学習を実行します．本学習前のデバッグ等において，この機能を使うことで開発効率を上げることが可能です．

In [ ]:
import subprocess

from sagemaker.chainer.estimator import Chainer

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

chainer_estimator = Chainer(entry_point='chainer_mnist.py', role=role,
                            train_instance_count=1, train_instance_type=instance_type,
                            hyperparameters={'epochs': 3, 'batch_size': 128})

chainer_estimator.fit({'train': train_input, 'test': test_input})

# モデルの推論を実行


推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．こちらもインスタンスタイプを `local` にすることで，このインスタンス内にエンドポイントを作成します．

In [ ]:
predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type=instance_type)

デプロイが終わったら，実際に手書き文字認識を行ってみましょう．

In [ ]:
import random

import matplotlib.pyplot as plt

num_samples = 5
indices = random.sample(range(test_images.shape[0] - 1), num_samples)
images, labels = test_images[indices], test_labels[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

In [ ]:
prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

あとは，上のキャンバスに文字を書いて，実際に予測をしてみましょう．

In [ ]:
image = np.array(data, dtype=np.float32)
prediction = predictor.predict(image)
predicted_label = prediction.argmax(axis=1)[0]
print('What you wrote is: {}'.format(predicted_label))

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
chainer_estimator.delete_endpoint()